# 🔄 GEM-B1: 순수 베이스라인 회귀 전략

**점수 회복을 위한 단순화:**
- ❌ TF-IDF 키워드 프롬프트 제거
- ❌ N-Best Reranking 제거
- ❌ 동적 추론 설정 제거
- ✅ 학습과 동일한 입력 형태 유지
- ✅ #Person# 태그 보존
- ✅ 단순 beam search만 사용

**목표: 48점대 회복 → 49점대 도전**

In [1]:
import pandas as pd
import os
import re
import json
import yaml
import random
import numpy as np
import torch
from glob import glob
from tqdm import tqdm
from pprint import pprint
from rouge import Rouge

# Transformers & Torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    BartForConditionalGeneration, 
    BartConfig,
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    EarlyStoppingCallback
)

# -----------------------------------------------------------------------------
# 1. Configuration & Seed Setting (설정 및 시드 고정)
# -----------------------------------------------------------------------------

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 순수 베이스라인 설정 (복잡한 전처리 모두 제거)
CONF = {
    "general": {
        "data_path": "./data/",
        "model_name": "digit82/kobart-summarization",
        "output_dir": "./results_gem_b1_pure_baseline",
        "seed": 42
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 128,
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', 
                          '#Person5#', '#Person6#', '#Person7#', 
                          '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 15,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 32,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 3,
        "fp16": True,
        "load_best_model_at_end": True,
        "metric_for_best_model": "combined_score",
        "greater_is_better": True,
        "logging_dir": "./logs",
        "logging_strategy": "steps",
        "logging_steps": 100,
        "predict_with_generate": True,
        "generation_max_length": 128,
        "early_stopping_patience": 3,
        "report_to": "none"
    },
    "inference": {
        "result_path": "./prediction/",
        # 순수 베이스라인 추론 설정
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "max_length": 100,
        "num_beams": 4,
        "length_penalty": 1.0,
        "batch_size": 32,
    }
}

seed_everything(CONF['general']['seed'])
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print("=" * 80)
print("🔄 순수 베이스라인 모드:")
print("   - TF-IDF 프롬프트: 비활성화")
print("   - 동적 추론: 비활성화")
print("   - N-Best Reranking: 비활성화")
print("   - 학습-추론 일관성: 최우선")
print("=" * 80)

# -----------------------------------------------------------------------------
# 2. Data Preprocessing (데이터 전처리)
# -----------------------------------------------------------------------------

# Tokenizer 로드 및 Special Token 추가
tokenizer = AutoTokenizer.from_pretrained(CONF['general']['model_name'])
special_tokens_dict = {'additional_special_tokens': CONF['tokenizer']['special_tokens']}
tokenizer.add_special_tokens(special_tokens_dict)

CONF['tokenizer']['bos_token'] = tokenizer.bos_token
CONF['tokenizer']['eos_token'] = tokenizer.eos_token

class Preprocess:
    def __init__(self, bos_token, eos_token):
        self.bos_token = bos_token
        self.eos_token = eos_token

    def make_set_as_df(self, file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[['fname', 'dialogue', 'summary']]
        else:
            return df[['fname', 'dialogue']]

    def make_input(self, dataset, is_test=False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

class CustomDataset(Dataset):
    def __init__(self, encoder_input, decoder_input, labels=None, is_test=False, ids=None):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.is_test = is_test
        self.ids = ids

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        if 'input_ids' in self.decoder_input:
            item['decoder_input_ids'] = self.decoder_input['input_ids'][idx].clone().detach()
            item['decoder_attention_mask'] = self.decoder_input['attention_mask'][idx].clone().detach()

        if not self.is_test:
            item['labels'] = self.labels['input_ids'][idx].clone().detach()
            
        if self.ids is not None:
            item['ID'] = self.ids[idx]
            
        return item

    def __len__(self):
        return len(self.encoder_input['input_ids'])

def prepare_data(conf, tokenizer, is_train=True):
    preprocessor = Preprocess(conf['tokenizer']['bos_token'], conf['tokenizer']['eos_token'])
    data_path = conf['general']['data_path']
    
    if is_train:
        train_df = preprocessor.make_set_as_df(os.path.join(data_path, 'train.csv'))
        val_df = preprocessor.make_set_as_df(os.path.join(data_path, 'dev.csv'))
        
        # Train Data
        enc_train, dec_in_train, dec_out_train = preprocessor.make_input(train_df)
        tokenized_enc_train = tokenizer(enc_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_train = tokenizer(dec_in_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        tokenized_dec_out_train = tokenizer(dec_out_train, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        train_dataset = CustomDataset(tokenized_enc_train, tokenized_dec_in_train, tokenized_dec_out_train)
        
        # Val Data
        enc_val, dec_in_val, dec_out_val = preprocessor.make_input(val_df)
        tokenized_enc_val = tokenizer(enc_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_val = tokenizer(dec_in_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        tokenized_dec_out_val = tokenizer(dec_out_val, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        val_dataset = CustomDataset(tokenized_enc_val, tokenized_dec_in_val, tokenized_dec_out_val)
        
        return train_dataset, val_dataset
    
    else: # Test
        test_df = preprocessor.make_set_as_df(os.path.join(data_path, 'test.csv'), is_train=False)
        enc_test, dec_in_test = preprocessor.make_input(test_df, is_test=True)
        
        tokenized_enc_test = tokenizer(enc_test, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['encoder_max_len'])
        tokenized_dec_in_test = tokenizer(dec_in_test, return_tensors="pt", padding=True, truncation=True, max_length=conf['tokenizer']['decoder_max_len'])
        
        test_dataset = CustomDataset(tokenized_enc_test, tokenized_dec_in_test, is_test=True, 
                                     ids=test_df['fname'].tolist())
        return test_df, test_dataset

# -----------------------------------------------------------------------------
# 3. Model Training (모델 학습)
# -----------------------------------------------------------------------------
def compute_metrics(eval_pred):
    rouge = Rouge()
    predictions, labels = eval_pred
    
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # skip_special_tokens=False로 #Person# 태그 보존
    decoded_preds = tokenizer.batch_decode(predictions.tolist(), skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=False)
    
    # 시스템 토큰만 제거 (#Person# 보존)
    remove_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']
    
    def clean_text(text_list):
        cleaned = []
        for text in text_list:
            for token in remove_tokens:
                if token is not None:
                    text = text.replace(token, "").strip() 
            cleaned.append(text)
        return cleaned

    decoded_preds = clean_text(decoded_preds)
    decoded_labels = clean_text(decoded_labels)
    
    try:
        results = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
        r1 = results["rouge-1"]["f"]
        r2 = results["rouge-2"]["f"]
        rl = results["rouge-l"]["f"]
        
        combined_score = (r1 + r2 + rl) / 3
        
        return {
            "rouge-1": r1,
            "rouge-2": r2,
            "rouge-l": rl,
            "combined_score": combined_score
        }
    except Exception as e:
        return {"combined_score": 0.0}


def train():
    print("\n" + "=" * 80)
    print("📚 모델 학습 시작 (순수 베이스라인)")
    print("=" * 80)
    
    # 데이터 로드
    train_dataset, val_dataset = prepare_data(CONF, tokenizer, is_train=True)
    
    # 모델 로드
    model = BartForConditionalGeneration.from_pretrained(CONF['general']['model_name'])
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    
    print(f"✅ 모델 로드 완료: {CONF['general']['model_name']}")
    print(f"   Vocab size: {len(tokenizer)}")
    
    # Training Arguments
    args = Seq2SeqTrainingArguments(
        output_dir=CONF['general']['output_dir'],
        overwrite_output_dir=CONF['training']['overwrite_output_dir'],
        num_train_epochs=CONF['training']['num_train_epochs'],
        learning_rate=CONF['training']['learning_rate'],
        per_device_train_batch_size=CONF['training']['per_device_train_batch_size'],
        per_device_eval_batch_size=CONF['training']['per_device_eval_batch_size'],
        warmup_ratio=CONF['training']['warmup_ratio'],
        weight_decay=CONF['training']['weight_decay'],
        lr_scheduler_type=CONF['training']['lr_scheduler_type'],
        optim=CONF['training']['optim'],
        evaluation_strategy=CONF['training']['evaluation_strategy'],
        save_strategy=CONF['training']['save_strategy'],
        save_total_limit=CONF['training']['save_total_limit'],
        fp16=CONF['training']['fp16'],
        load_best_model_at_end=CONF['training']['load_best_model_at_end'],
        metric_for_best_model=CONF['training']['metric_for_best_model'],
        greater_is_better=CONF['training']['greater_is_better'],
        logging_dir=CONF['training']['logging_dir'],
        logging_steps=CONF['training']['logging_steps'],
        predict_with_generate=CONF['training']['predict_with_generate'],
        generation_max_length=CONF['training']['generation_max_length'],
        report_to=CONF['training']['report_to'],
        seed=CONF['general']['seed']
    )
    
    # Early Stopping
    early_stopping = EarlyStoppingCallback(
        early_stopping_patience=CONF['training']['early_stopping_patience']
    )
    
    # Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping]
    )
    
    print("\n🚀 Training Start...")
    trainer.train()
    
    # Best Model 저장
    best_model_path = os.path.join(CONF['general']['output_dir'], "best_model")
    trainer.save_model(best_model_path)
    print(f"\n✅ Training Finished. Best Model Saved at {best_model_path}")
    
    return best_model_path

# -----------------------------------------------------------------------------
# 4. Pure Baseline Inference (순수 베이스라인 추론)
# -----------------------------------------------------------------------------
def inference_pure_baseline(model_path=None):
    """
    순수 베이스라인 추론
    - 프롬프트 없음
    - 전처리 없음
    - 단순 beam search
    - #Person# 태그 보존
    """
    if model_path is None:
        model_path = os.path.join(CONF['general']['output_dir'], "best_model")
        
    print("\n" + "=" * 80)
    print(f"🔄 순수 베이스라인 추론")
    print(f"   Model: {model_path}")
    print("=" * 80)
    
    model = BartForConditionalGeneration.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    # 데이터 로드
    test_df, test_dataset = prepare_data(CONF, tokenizer, is_train=False)
    dataloader = DataLoader(test_dataset, batch_size=CONF['inference']['batch_size'], shuffle=False)
    
    summary_list = []
    fname_list = []
    
    # 시스템 토큰만 제거 (#Person# 보존)
    system_tokens = [tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, '<usr>']

    print("\n🚀 Inference Start...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # 단순 beam search
            summary_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                num_beams=CONF['inference']['num_beams'],
                max_length=CONF['inference']['max_length'],
                no_repeat_ngram_size=CONF['inference']['no_repeat_ngram_size'],
                early_stopping=CONF['inference']['early_stopping'],
                length_penalty=CONF['inference']['length_penalty']
            )
            
            # skip_special_tokens=False로 #Person# 보존
            decoded = tokenizer.batch_decode(summary_ids, skip_special_tokens=False)
            
            # 시스템 토큰만 제거
            cleaned_batch = []
            for text in decoded:
                for token in system_tokens:
                    if token is not None:
                        text = text.replace(token, "").strip()
                # 연속 공백 제거
                text = ' '.join(text.split())
                cleaned_batch.append(text)
            
            summary_list.extend(cleaned_batch)
            fname_list.extend(batch['ID'])
    
    # 결과 저장
    result_path = CONF['inference']['result_path']
    os.makedirs(result_path, exist_ok=True)
    
    output_df = pd.DataFrame({
        "fname": fname_list,
        "summary": summary_list
    })
    
    save_file = os.path.join(result_path, "submit_gem_b1_pure_baseline.csv")
    output_df.to_csv(save_file, index=False)
    print(f"\n✅ Inference Finished. Saved at {save_file}")
    
    # 통계 출력
    lengths = [len(s) for s in summary_list]
    print(f"\n📊 요약문 통계:")
    print(f"   평균 길이: {np.mean(lengths):.1f}자")
    print(f"   최소/최대: {min(lengths)}/{max(lengths)}자")
    
    # 샘플 출력
    print(f"\n📝 샘플 요약 (처음 3개):")
    for i in range(min(3, len(summary_list))):
        print(f"   [{i+1}] {summary_list[i][:100]}...")
    
    return output_df


# -----------------------------------------------------------------------------
# 5. Main Execution
# -----------------------------------------------------------------------------

if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("🔄 GEM-B1: 순수 베이스라인 회귀 전략")
    print("=" * 80)
    print("\n목표:")
    print("  1. 모든 복잡한 전처리 제거")
    print("  2. 학습-추론 일관성 최우선")
    print("  3. #Person# 태그 보존")
    print("  4. 단순 beam search만 사용")
    print("  5. 점수 48점대 회복")
    print("=" * 80)
    
    # 1. 학습 실행
    best_ckpt = train()
    
    # 2. 순수 베이스라인 추론
    output = inference_pure_baseline(best_ckpt)
    
    print("\n" + "=" * 80)
    print("✅ 모든 작업 완료!")
    print("=" * 80)
    print(f"\n생성된 제출 파일:")
    print(f"  - submit_gem_b1_pure_baseline.csv")
    print(f"\n💡 이 파일을 리더보드에 제출하여 점수 회복을 확인하세요!")
    print(f"   예상 점수: 48점대")
    print("=" * 80)

Device: cuda:0
🔄 순수 베이스라인 모드:
   - TF-IDF 프롬프트: 비활성화
   - 동적 추론: 비활성화
   - N-Best Reranking: 비활성화
   - 학습-추론 일관성: 최우선

🔄 GEM-B1: 순수 베이스라인 회귀 전략

목표:
  1. 모든 복잡한 전처리 제거
  2. 학습-추론 일관성 최우선
  3. #Person# 태그 보존
  4. 단순 beam search만 사용
  5. 점수 48점대 회복

📚 모델 학습 시작 (순수 베이스라인)

🔄 GEM-B1: 순수 베이스라인 회귀 전략

목표:
  1. 모든 복잡한 전처리 제거
  2. 학습-추론 일관성 최우선
  3. #Person# 태그 보존
  4. 단순 beam search만 사용
  5. 점수 48점대 회복

📚 모델 학습 시작 (순수 베이스라인)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


✅ 모델 로드 완료: digit82/kobart-summarization
   Vocab size: 30010

🚀 Training Start...


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 268.94 MiB is free. Process 2610232 has 3.15 GiB memory in use. Process 2795486 has 14.77 GiB memory in use. Process 3253811 has 5.50 GiB memory in use. Of the allocated memory 5.12 GiB is allocated by PyTorch, and 74.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# -----------------------------------------------------------------------------
# 기존 학습된 모델로 순수 베이스라인 추론 (메모리 효율적)
# -----------------------------------------------------------------------------

print("\n" + "=" * 80)
print("🔄 기존 학습 모델로 순수 베이스라인 추론")
print("=" * 80)

# 기존 모델 경로
MODEL_PATH = "./prediction_kobart_v1/checkpoint-4674"

print(f"\n✅ 기존 모델 사용: {MODEL_PATH}")
print("   메모리 부족으로 재학습 대신 기존 모델 활용")
print("   모든 전처리 제거하고 순수 추론만 수행")

# 순수 베이스라인 추론 실행
output = inference_pure_baseline(MODEL_PATH)

print("\n" + "=" * 80)
print("✅ 순수 베이스라인 추론 완료!")
print("=" * 80)
print(f"\n생성된 제출 파일:")
print(f"  - submit_gem_b1_pure_baseline.csv")
print(f"\n💡 리더보드에 제출하여 점수 회복을 확인하세요!")
print(f"   예상: 48점대로 회복 (이전 39.67점 → 48점대)")
print("=" * 80)